In [ ]:
%matplotlib inline
import os
from collections import namedtuple
from collections import defaultdict
from collections import Counter
from datetime import datetime
import itertools
import base64
import glob
import json
import random
import time
import imageio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import shutil
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import confusion_matrix
import boto3
import botocore
import sagemaker
from urllib.parse import urlparse


In [ ]:
!wget -O data.zip https://tinyurl.com/yatr4655
!unzip data.zip    

In [ ]:
import boto3, sagemaker

region = boto3.session.Session().region_name
session = sagemaker.session.Session()
bucket = session.default_bucket()

In [ ]:
!aws s3 cp --recursive FudanPed_gt_demo s3://$bucket/FudanPed_gt_demo/

In [ ]:
import os 
import json 

manifest_name = 'manifest.jsonl'
manifestfile = open(manifest_name, 'w')
EXP_NAME = 'FudanPed_gt_demo'
for f in os.listdir("FudanPed_gt_demo"):
    img_path = "s3://{}/FudanPed_gt_demo/{}".format(bucket, f)
    line = '{"source-ref": "' + img_path + '"}\n'
    manifestfile.write(line)
manifestfile.close()    

!aws s3 cp  manifest.jsonl s3://$bucket/FudanPed_gt_demo/

In [ ]:

CLASS_NAME = "HUMAN"
CLASS_LIST = [CLASS_NAME]
print("Label space is {}".format(CLASS_LIST))
s3 = boto3.client('s3')
json_body = {"labels": [{"label": label} for label in CLASS_LIST]}
with open("class_labels.json", "w") as f:
    json.dump(json_body, f)

s3.upload_file("class_labels.json", bucket, "FudanPed_gt_demo/class_labels.json")


### Configure CORS of our bucket 

* make sure the bucket have CORS enabled 
```
[{
   "AllowedHeaders": [],
   "AllowedMethods": ["GET"],
   "AllowedOrigins": ["*"],
   "ExposeHeaders": []
}]
```

In [ ]:
cors_configuration = {
    'CORSRules':[{
     "AllowedHeaders": [],
     "AllowedMethods": ["GET"],
     "AllowedOrigins": ["*"],
     "ExposeHeaders": []
    }]
}

# Set the CORS configuration

s3.put_bucket_cors(Bucket=bucket,
                   CORSConfiguration=cors_configuration)

### Configure groundtruth workforce 

In [ ]:
private_workteam_arn = "arn:aws:sagemaker:us-west-2:294738042744:workteam/private-crowd/seal-squad"

### Configure groundtruth job

In [ ]:


from IPython.core.display import HTML, display


def make_template(test_template=False, save_fname="instructions.template"):
    template = r"""<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
<crowd-form>
  <crowd-semantic-segmentation
    name="crowd-semantic-segmentation"
    src="{{ task.input.taskObject | grant_read_access }}"
    header="Please segment out all pedestrians."
    labels="{{ task.input.labels | to_json | escape }}"
  >
    <full-instructions header="Segmentation instructions">
      <ol><li><strong>Read</strong> the task carefully and inspect the image.</li>
      <li><strong>Read</strong> the options and review the examples provided to understand more about the labels.</li>
      <li><strong>Choose</strong> the appropriate label that best suits an object and paint that object using the tools provided.</li></ol>
    </full-instructions>
    <short-instructions>
      <h2><span style="color: rgb(0, 138, 0);">Good example</span></h2>
      <p>Enter description to explain a correctly done segmentation</p>
      <p><br></p><h2><span style="color: rgb(230, 0, 0);">Bad example</span></h2>
      <p>Enter description of an incorrectly done segmentation</p>
    </short-instructions>
  </crowd-semantic-segmentation>
</crowd-form>
    """
    with open(save_fname, "w") as f:
        f.write(template)


make_template(test_template=True, save_fname="instructions.html")
make_template(test_template=False, save_fname="instructions.template")
s3.upload_file("instructions.template", bucket, "FudanPed_gt_demo/instructions.template")

In [ ]:
# Specify ARNs for resources needed to run an object detection job.
ac_arn_map = {
    "us-west-2": "081040173940",
    "us-east-1": "432418664414",
    "us-east-2": "266458841044",
    "eu-west-1": "568282634449",
    "ap-northeast-1": "477331159723",
}

prehuman_arn = "arn:aws:lambda:{}:{}:function:PRE-SemanticSegmentation".format(region, ac_arn_map[region])
acs_arn = "arn:aws:lambda:{}:{}:function:ACS-SemanticSegmentation".format(region, ac_arn_map[region])
# labeling_algorithm_specification_arn = "arn:aws:sagemaker:{}:027400017018:labeling-job-algorithm-specification/object-detection".format(
#     region
# )

In [ ]:
role = sagemaker.get_execution_role()


task_description = "Dear Annotator, please segment around each {}. Thank you!".format(CLASS_NAME)
task_keywords = ["image", "semantic", "segmentation"]
task_title = "Please draw a box around each {}.".format(CLASS_NAME)
job_name = "human-segmentation-" + str(int(time.time()))

human_task_config = {
    "AnnotationConsolidationConfig": {
        "AnnotationConsolidationLambdaArn": acs_arn,
    },
    "PreHumanTaskLambdaArn": prehuman_arn,
    "MaxConcurrentTaskCount": 200,  # 200 images will be sent at a time to the workteam.
    "NumberOfHumanWorkersPerDataObject": 1,  # We will obtain and consolidate 5 human annotations for each image.
    "TaskAvailabilityLifetimeInSeconds": 21600,  # Your workteam has 6 hours to complete all pending tasks.
    "TaskDescription": task_description,
    "TaskKeywords": task_keywords,
    "TaskTimeLimitInSeconds": 300,  # Each image must be labeled within 5 minutes.
    "TaskTitle": task_title,
    "UiConfig": {
        "UiTemplateS3Uri": "s3://{}/{}/instructions.template".format(bucket, EXP_NAME),
    },
}


human_task_config["WorkteamArn"] = private_workteam_arn

ground_truth_request = {
    "InputConfig": {
        "DataSource": {
            "S3DataSource": {
                "ManifestS3Uri": "s3://{}/{}/{}".format(bucket, EXP_NAME, manifest_name),
            }
        },
        "DataAttributes": {
            "ContentClassifiers": ["FreeOfPersonallyIdentifiableInformation", "FreeOfAdultContent"]
        },
    },
    "OutputConfig": {
        "S3OutputPath": "s3://{}/{}/output/".format(bucket, EXP_NAME),
    },
    "HumanTaskConfig": human_task_config,
    "LabelingJobName": job_name,
    "RoleArn": role,
    "LabelAttributeName": "category-ref",
    "LabelCategoryConfigS3Uri": "s3://{}/{}/class_labels.json".format(bucket, EXP_NAME),
}



sagemaker_client = boto3.client("sagemaker")
sagemaker_client.create_labeling_job(**ground_truth_request)

In [ ]:
# !aws s3 cp  s3://sagemaker-us-west-2-294738042744/FudanPed_gt_demo/output/human-segmentation-1622688978/manifests/output/output.manifest . 
!aws s3 cp  <s3 uri of manifest file> . 

In [ ]:
!cat output.manifest

In [ ]:
import json 
import boto3 
import cv2 
from IPython.display import Image 

s3 = boto3.client('s3')
output_file = open("output.manifest", "r")
lines = output_file.readlines()
for line in lines: 
    obj = json.loads(line)
    s3_output = obj['category-ref'] 
    tokens = s3_output.split(bucket)
    file_name = tokens[-1].split('/')[-1]
    s3.download_file(bucket, tokens[1][1:], file_name)
    img = Image(file_name)
    display(img)
    
    cv2img = cv2.imread(file_name)
    print(cv2img.shape)
